In [56]:
import numpy as np
from tools import Data
from collections import Counter
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
start = time.time()
data = Data()
print(time.time() - start)

Calculating Vocabulary...
Loading Testing Data...
Loading Categories...
Loading Training Data...
239.75982117652893
